In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
print("Libraries imported successfully")

Libraries imported successfully


In [14]:
path =r"C:\Users\Admin\OneDrive\Рабочий стол\house price prediction\malaysia_house_price_data_2025.csv"
df=pd.read_csv(path)
print("Shape of dataset:", df.shape)
display(df.head(5))
display(df.dtypes)
print("\nMissing values per column(top 10): ")
print(df.isna().sum().sort_values(ascending=False).head(10))

Shape of dataset: (2000, 8)


,Township,Area,State,Tenure,Type,Median_Price,Median_PSF,Transactions
0,SCIENTEX SUNGAI DUA,Tasek Gelugor,Penang,Freehold,Terrace House,331800.0,304.0,593
1,BANDAR PUTRA,Kulai,Johor,Freehold,"Cluster House, Terrace House",590900.0,322.0,519
2,TAMAN LAGENDA TROPIKA TAPAH,Chenderiang,Perak,Freehold,Terrace House,229954.0,130.0,414
3,SCIENTEX JASIN MUTIARA,Bemban,Melaka,Freehold,Terrace House,255600.0,218.0,391
4,TAMAN LAGENDA AMAN,Tapah,Perak,Leasehold,Terrace House,219300.0,168.0,363


Township         object
Area             object
State            object
Tenure           object
Type             object
Median_Price    float64
Median_PSF      float64
Transactions      int64
dtype: object


Missing values per column(top 10): 
Township        0
Area            0
State           0
Tenure          0
Type            0
Median_Price    0
Median_PSF      0
Transactions    0
dtype: int64


In [17]:
X=df.drop("Median_Price", axis=1)
y=df["Median_Price"]
categorical_cols= X.select_dtypes(include=['object']).columns.tolist()
numerical_cols=X.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer= StandardScaler()
categorical_transformer=OneHotEncoder(handle_unknown='ignore')
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numeric_transformer,numerical_cols),
        ('cat', categorical_transformer,categorical_cols)
    ])

from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(
    X,y,test_size=0.2,random_state=42)
print("Training data shape:", X_train.shape)
print("Test data shape: ", X_test.shape)


Categorical columns: ['Township', 'Area', 'State', 'Tenure', 'Type']
Numerical columns: ['Median_PSF', 'Transactions']
Training data shape: (1600, 7)
Test data shape:  (400, 7)


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
model = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor', Ridge(alpha=1.0))
])

model.fit(X_train,y_train)
y_pred=model.predict(X_test)

from sklearn.metrics import mean_squared_error,r2_score
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print("Mean Squared Error:", mse)
print("R2 Score:",r2)

Mean Squared Error: 106871320104.99254
R2 Score: 0.39249795412969


In [22]:
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

categorical_cols=X.select_dtypes(include =['object']).columns.tolist()
numerical_cols=X.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_transformer=StandardScaler()
categorical_transformer=OneHotEncoder(handle_unknown='ignore')
preprocessor=ColumnTransformer([
    ('num', numeric_transformer,numerical_cols),
    ('cat', categorical_transformer,categorical_cols)
])

lgb_model=Pipeline([
    ('preprocessor',preprocessor),
    ('regressor', lgb.LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=10,
        random_state=42
    ))
])

In [24]:
lgb_model.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000138 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 427
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 44
[LightGBM] [Info] Start training from score 492318.251875
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [25]:
y_pred=lgb_model.predict(X_test)


In [26]:
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print("Mean Squared Error: ", mse)
print("R2 Score:",r2)

Mean Squared Error:  119688540422.39586
R2 Score: 0.31963942147992563


In [ ]:
categorical_cols = ['Township', 'Area', 'State', 'Tenure', 'Type']
for col in categorical_cols:
    df[col] = df[col].astype('category')

df['Estimated_Total_Price'] = df['Median_PSF'] * df['Transactions']

X = df.drop("Median_Price", axis=1)
y = df["Median_Price"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score


lgbm_model = LGBMRegressor(
    n_estimators=2000,       
    learning_rate=0.03,      
    max_depth=15,            
    num_leaves=40,           
    random_state=42,
    verbose=-1
)

lgbm_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    categorical_feature=categorical_cols,
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
)

# Step 7: Predict & evaluate
y_pred = lgbm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[46]	valid_0's rmse: 266699	valid_0's l2: 7.11285e+10
Mean Squared Error: 71128468128.73666
R2 Score: 0.5956755295492036
